## Magdalena Augustyńska

**Problem 5b (Testing a sampler).** In this problem we will attempt to check whether the sampler we created in **Problem 2c** works correctly. To this end we will use a chi-squared goodness-of-fit test. This test works as follows:
 * Let $p_1,\ldots,p_d$ be the date frequencies as in the text file, scaled down to sum up to 1.
 * Use the sampler to generate a sample of dates. Let $c_1,\ldots,c_d$ be the observed counts, and let $f_i=Np_i$ be the expected counts, where $N$ is the sample size. 
 * Compute the test statistic $$S = \sum_{i=1}^d \frac{\left(c_i-f_i\right)^2}{f_i}.$$
 * Our base assumption (the null hypothesis) $H_0$ is that our sampler works correctly. If $H_0$ is true AND if the expected count for each bucket is large enough, then $S$ has (approximately) a $\chi^2$ distribution with $d-1$ degrees of freedom. 
 * Look up how likely is getting an $S$ value as large as the one you obtained if it has that distribution, i.e. the $p$-value. To do this use **scipy.stats.chi2.cdf**. If this value turns out smaller than the assumed threshold, e.g. $0.05$, we reject $H_0$. Otherwise we do not (we support $H_0$), but this does not mean $H_0$ is proved!
 * We mentioned earlier that expected counts for the buckets need to be large enough. "Large enough" assumption here is used to guarantee that $c_i$ are distributed approximately normally. Typically one requires that all counts are at least $5$. This is not the case in our problem (unless we take a huge sample) because of the errors in the data. The typical approach is to glue several buckets into one but this does not help in our case. Instead, ignore the erroneous dates when computing $c_i$ and $f_i$ and run the test again (on the same sample!). Remember to use a different number of degrees of freedom. Compare the results. 
 * Perform the same test using **scipy.stats.chisquare** and compare the results.

#### Let $p_1,\ldots,p_d$ be the date frequencies as in the text file, scaled down to sum up to 1.

In [1]:
import numpy as np
import math
import pandas as pd
from random import randint
import time
from collections import Counter

In [2]:
data = pd.read_csv('../../rpis/lab1/us_births_69_88.csv')

In [3]:
all_births = sum(data['births'])

In [4]:
freq = [data['births'][i]/all_births for i in range(data.shape[0])]

In [5]:
d = len(freq)

#### Use the sampler to generate a sample of dates. Let $c_1,\ldots,c_d$ be the observed counts, and let $f_i=Np_i$ be the expected counts, where $N$ is the sample size. 

In [6]:
def simulate(probabilities):
    V = np.mean(probabilities)
    days = len(probabilities)
    buckets = np.empty(days, dtype=list)
    for i in range(days):
        bucket = [[i+1], [probabilities[i]]]
        buckets[i] = bucket 
                          
    while True:
        A_index = next((index for index, value in enumerate(buckets) if len(value[0]) == 1 and value[1][0] < V), None)
        B_index = next((index for index, value in enumerate(buckets) if len(value[0]) == 1 and value[1][0] > V), None)
        if A_index is None or B_index is None:
            break
        to_transfer = V - buckets[A_index][1][0]
        buckets[A_index][1].append(to_transfer)
        buckets[A_index][0].append(buckets[B_index][0][0])
        buckets[B_index][1][0] -= to_transfer
    return buckets

In [7]:
def sample_from_buckets_vec(buckets, size):
    days_numbers = np.random.randint(len(buckets), size=size)
    result = np.empty(size)
    sampled_days = buckets[days_numbers]
    prob = [sampled_days[i][1]/sum(sampled_days[i][1]) for i in range(size)]
    for i in range(len(prob)):
        if prob[i].shape[0] == 1:
            prob[i] = np.pad(prob[i], pad_width=(0, 1), mode='constant')
    prob = np.array(prob)
    days = np.array([sampled_days[i][0] for i in range(size)])
    c = prob.cumsum(axis=1)
    u = np.random.rand(len(c), 1)
    choices = (u < c).argmax(axis=1)
    result = [days[i][choice] for i, choice in enumerate(choices)]
    return result

In [8]:
def random_date_buckets_vec(buckets):
    step = 0
    occ = [0] * len(buckets)
    random_days = sample_from_buckets_vec(buckets, 10)
    while True:
        if step%10 == 0:
            random_days = sample_from_buckets_vec(buckets, 10)
        day = random_days[step%10]
        step += 1
        if (occ[day - 1]) == 1:
            return step
        else:
            occ[day-1] += 1

In [9]:
buckets = simulate(freq)

In [10]:
N = 10000

In [11]:
result_buckets = [random_date_buckets_vec(buckets) for i in range(N)]

In [12]:
counts = Counter(result_buckets)

In [13]:
def calculate_expected(frequencies, sample_size):
    return [sample_size*p for p in frequencies]

In [14]:
def calculate_observed(size, counter):
    observed = []
    for i in range(size):
        observed.append(counter[i + 1])
    return observed

In [15]:
expected = calculate_expected(freq, N)

In [16]:
observed = calculate_observed(d, counts)

#### Compute the test statistic $$S = \sum_{i=1}^d \frac{\left(c_i-f_i\right)^2}{f_i}.$$

In [17]:
def compute_test_stat(observed, expected):
    S = 0
    for i in range(len(expected)):
        c_i = observed[i]
        f_i = expected[i]
        S += (c_i - f_i)**2/f_i
    return S

In [18]:
statistic = compute_test_stat(counts, expected)
statistic

98754.73918695652

#### Our base assumption (the null hypothesis) $H_0$ is that our sampler works correctly. If $H_0$ is true AND if the expected count for each bucket is large enough, then $S$ has (approximately) a $\chi^2$ distribution with $d-1$ degrees of freedom. 

#### Look up how likely is getting an $S$ value as large as the one you obtained if it has that distribution, i.e. the $p$-value. To do this use **scipy.stats.chi2.cdf**. If this value turns out smaller than the assumed threshold, e.g. $0.05$, we reject $H_0$. Otherwise we do not (we support $H_0$), but this does not mean $H_0$ is proved!

In [19]:
from scipy.stats import chi2

In [20]:
p_value = 1.0 - chi2.cdf(statistic, d - 1)

In [21]:
p_value

0.0

The $p$-value turns out smaller than the assumed threshold, e.g. $0.05$, therefore we reject $H_0$. 

#### We mentioned earlier that expected counts for the buckets need to be large enough. "Large enough" assumption here is used to guarantee that $c_i$ are distributed approximately normally. Typically one requires that all counts are at least $5$. This is not the case in our problem (unless we take a huge sample) because of the errors in the data. The typical approach is to glue several buckets into one but this does not help in our case. Instead, ignore the erroneous dates when computing $c_i$ and $f_i$ and run the test again (on the same sample!). Remember to use a different number of degrees of freedom. Compare the results. 

We remove rows representing days that don't exist int he calendar.

In [22]:
data = pd.read_csv('../../rpis/lab1/us_births_69_88.csv')
for i in [2, 4, 6, 9, 11]:
    data.drop(data.index[(data['month'] == i) & (data['day'] == 31)], inplace = True)
    
data.drop(data.index[(data['month'] == 2) & (data['day'] == 30)], inplace = True)
data.index = range(len(data))

In [23]:
all_births = sum(data['births'])
freq = [data['births'][i]/all_births for i in range(data.shape[0])]
d = len(freq)

In [24]:
buckets = simulate(freq)

In [25]:
result_buckets = [random_date_buckets_vec(buckets) for i in range(N)]

In [26]:
counts = Counter(result_buckets)

In [27]:
expected = calculate_expected(freq, N)

In [28]:
observed = calculate_observed(d, counts)

In [29]:
statistic = compute_test_stat(observed, expected)
statistic

79062.48783165358

In [30]:
p_value = 1.0 - chi2.cdf(statistic, d - 1)

In [31]:
p_value

0.0

The $p$-value ia again smaller than the assumed threshold, e.g. $0.05$, therefore we reject $H_0$. 

#### Perform the same test using **scipy.stats.chisquare** and compare the results.

In [32]:
from scipy.stats import chisquare

In [33]:
p_value = chisquare(observed, expected)[1]

In [34]:
p_value

0.0

The $p$-value ia again equal 0 and we reject $H_0$. 